# 0.0: pull packages

In [60]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import joblib
import numpy as np
import random
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model, model_selection, ensemble, svm, naive_bayes, preprocessing
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, r2_score, mean_absolute_error, mean_squared_error
from sklearn.decomposition import FactorAnalysis
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import getpass
import pyodbc
import sys
import sqlalchemy
import urllib
from datetime import date
from collections import Counter
import xgboost as xgb
from xgboost.sklearn import XGBClassifier, XGBRegressor
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4
import seaborn as sns
import time
from catboost import Pool, CatBoostClassifier, CatBoostRegressor
import catboost
import shap
from sklearn.impute import SimpleImputer

In [61]:
pd.set_option("display.max_columns",100)
pd.set_option("display.max_rows",100)

## 0.1: read data

In [62]:
H09 = ['HealthDeterminantsDB']

In [63]:
for i in H09:
    if i == H09[0]:
        params = urllib.parse.quote_plus('DSN=HW2WIPSQL09;DATABASE=%s;Trusted_Connection=yes' %i)
        engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

        result = engine.execute('SELECT DISTINCT * FROM DS_SDOHScoring_002_sdoh')
        df_sdoh = pd.DataFrame(result.fetchall(),columns=result.keys())
        

In [64]:
df_sdoh.head()

,topic,subset,subset2,subset3,subset4,subset5,subset6,ZIP,county_name,state_abbrev,value,measure_type
0,Clinical Care,Access to Care,Dentists,# Dentists,,,,,ABBEVILLE,SC,3,Estimate
1,Clinical Care,Access to Care,Dentists,# Dentists,,,,,ACADIA,LA,26,Estimate
2,Clinical Care,Access to Care,Dentists,# Dentists,,,,,ACCOMACK,VA,15,Estimate
3,Clinical Care,Access to Care,Dentists,# Dentists,,,,,ADA,ID,376,Estimate
4,Clinical Care,Access to Care,Dentists,# Dentists,,,,,ADAIR,IA,4,Estimate


In [65]:
df_sdoh['sdoh_feature']=df_sdoh['topic']+' '+df_sdoh['subset']+' '+df_sdoh['subset2']+' '+df_sdoh['subset3']+' '+df_sdoh['subset4']+' '+df_sdoh['subset5']+' '+df_sdoh['subset6']

In [66]:
df_sdoh_pivot = pd.pivot_table(df_sdoh, values='value', index=['ZIP','county_name','state_abbrev'], columns=['sdoh_feature'])
df_sdoh_pivot=df_sdoh_pivot.reset_index()
df_sdoh_pivot=df_sdoh_pivot.rename(columns=lambda x: x.strip())

In [67]:
df_sdoh_pivot.head()

sdoh_feature,ZIP,county_name,state_abbrev,COMMUTING TO WORK Workers 16 years and over Public transportation (excluding taxicab),Clinical Care Access to Care Dentists # Dentists,Clinical Care Access to Care Mental Health Providers # Mental Health Providers,Clinical Care Access to Care PCPs # Primary Care Physicians,Clinical Care Access to Care Uninsured % Uninsured,Clinical Care Quality of Care Flu Vaccinations % Vaccinated for Flu,Clinical Care Quality of Care Mammography Screening % With Annual Mammogram,Demographics % Rural,Health Behaviors Alcohol and Drug Use Alcohol-impaired driving deaths % Driving Deaths with Alcohol Involvement,Health Behaviors Alcohol and Drug Use Drug Overdose Deaths Drug Overdose Mortality Rate,Health Behaviors Alcohol and Drug Use Excessive Drinking % Excessive Drinking,Health Behaviors Alcohol and Drug Use Motor Vehicle Crash Deaths Motor Vehicle Mortality Rate,Health Behaviors Diet and Exercise Adult Obesity % Adults with Obesity,Health Behaviors Diet and Exercise Food Environment Index Food Environment Index,Health Behaviors Diet and Exercise Food Insecurity % Food Insecure,Health Behaviors Diet and Exercise Physical Inactivity % Physically Inactive,Health Behaviors Other Health Behaviors Insufficient Sleep % Insufficient Sleep,Health Behaviors Other Health Behaviors Mental Health % Insufficient Sleep,Health Behaviors Sexual Activity STIs # Chlamydia Cases,Health Behaviors Sexual Activity Teen Births Teen Birth Rate,Health Behaviors Tobacco Use Adult Smoking % Smokers,Health Outcomes Length of Life Child Mortality Child Mortality Rate,Health Outcomes Length of Life Infant Mortality Infant Mortality Rate,Health Outcomes Length of Life Life Expectancy Life Expectancy,Health Outcomes Length of Life Premature Death Years of Potential Life Lost Rate,Health Outcomes Quality of Life Diabetes % Adults with Diabetes,Health Outcomes Quality of Life Mental Health % Frequent Mental Distress,Health Outcomes Quality of Life Mental Health Average Number of Mentally Unhealthy Days,Health Outcomes Quality of Life Physical Health % Frequent Physical Distress,Health Outcomes Quality of Life Physical Health Average Number of Physically Unhealthy Days,Health Outcomes Quality of Life STIs HIV Prevalence Rate,INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households Median household income (dollars),INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households With Food Stamp/SNAP benefits in the past 12 months,INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households With Social Security Mean Social Security income (dollars),INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households With retirement income Mean retirement income (dollars),Physical Environment Air and Water Quality Air Pollution Average Daily PM2.5,Physical Environment Housing and Transit Homeownership % Homeowners,Physical Environment Housing and Transit Severe Housing Cost Burden % Severe Housing Cost Burden,Physical Environment Housing and Transit Severe Housing Problems % Severe Housing Problems,Social and Economic Factors Community Safety Firearm Fatalities Firearm Fatalities Rate,Social and Economic Factors Community Safety Homicides Homicide Rate,Social and Economic Factors Community Safety Injury Deaths Injury Death Rate,Social and Economic Factors Community Safety Juvenile Arrests Juvenile Arrest Rate,Social and Economic Factors Community Safety Suicides Suicide Rate (Age-Adjusted),Social and Economic Factors Community Safety Violent Crime Violent Crime Rate,Social and Economic Factors Education Academic Performance Average Grade Performance,Social and Economic Factors Education Disconnected Youth % Disconnected Youth,Social and Economic Factors Education High School Graduation Rate High School Graduation Rate,Social and Economic Factors Education Some College % Some College,Social and Economic Factors Employment Unemployment % Unemployed,Social and Economic Factors Family and Social S

In [68]:
df_sdoh_pivot['sdoh_1']=df_sdoh_pivot['Social and Economic Factors Education High School Graduation Rate High School Graduation Rate'].rank(pct=True)*-1
df_sdoh_pivot['sdoh_2']=df_sdoh_pivot['Health Behaviors Alcohol and Drug Use Alcohol-impaired driving deaths % Driving Deaths with Alcohol Involvement'].rank(pct=True)
df_sdoh_pivot['sdoh_3']=df_sdoh_pivot['Social and Economic Factors Community Safety Suicides Suicide Rate (Age-Adjusted)'].rank(pct=True)
df_sdoh_pivot['sdoh_4']=df_sdoh_pivot['Clinical Care Access to Care Uninsured % Uninsured'].rank(pct=True)
df_sdoh_pivot['sdoh_5']=df_sdoh_pivot['Social and Economic Factors Family and Social Support Social Associations Social Association Rate'].rank(pct=True)*-1
df_sdoh_pivot['sdoh_6']=df_sdoh_pivot['Health Behaviors Diet and Exercise Food Environment Index Food Environment Index'].rank(pct=True)*-1
df_sdoh_pivot['sdoh_7']=df_sdoh_pivot['Clinical Care Quality of Care Mammography Screening % With Annual Mammogram'].rank(pct=True)*-1
df_sdoh_pivot['sdoh_8']=df_sdoh_pivot['Social and Economic Factors Income Income Inequality Ratio of 80th Percentile HH income to 20th Percentile HH Income'].rank(pct=True)

In [69]:
df_sdoh_pivot.head()

sdoh_feature,ZIP,county_name,state_abbrev,COMMUTING TO WORK Workers 16 years and over Public transportation (excluding taxicab),Clinical Care Access to Care Dentists # Dentists,Clinical Care Access to Care Mental Health Providers # Mental Health Providers,Clinical Care Access to Care PCPs # Primary Care Physicians,Clinical Care Access to Care Uninsured % Uninsured,Clinical Care Quality of Care Flu Vaccinations % Vaccinated for Flu,Clinical Care Quality of Care Mammography Screening % With Annual Mammogram,Demographics % Rural,Health Behaviors Alcohol and Drug Use Alcohol-impaired driving deaths % Driving Deaths with Alcohol Involvement,Health Behaviors Alcohol and Drug Use Drug Overdose Deaths Drug Overdose Mortality Rate,Health Behaviors Alcohol and Drug Use Excessive Drinking % Excessive Drinking,Health Behaviors Alcohol and Drug Use Motor Vehicle Crash Deaths Motor Vehicle Mortality Rate,Health Behaviors Diet and Exercise Adult Obesity % Adults with Obesity,Health Behaviors Diet and Exercise Food Environment Index Food Environment Index,Health Behaviors Diet and Exercise Food Insecurity % Food Insecure,Health Behaviors Diet and Exercise Physical Inactivity % Physically Inactive,Health Behaviors Other Health Behaviors Insufficient Sleep % Insufficient Sleep,Health Behaviors Other Health Behaviors Mental Health % Insufficient Sleep,Health Behaviors Sexual Activity STIs # Chlamydia Cases,Health Behaviors Sexual Activity Teen Births Teen Birth Rate,Health Behaviors Tobacco Use Adult Smoking % Smokers,Health Outcomes Length of Life Child Mortality Child Mortality Rate,Health Outcomes Length of Life Infant Mortality Infant Mortality Rate,Health Outcomes Length of Life Life Expectancy Life Expectancy,Health Outcomes Length of Life Premature Death Years of Potential Life Lost Rate,Health Outcomes Quality of Life Diabetes % Adults with Diabetes,Health Outcomes Quality of Life Mental Health % Frequent Mental Distress,Health Outcomes Quality of Life Mental Health Average Number of Mentally Unhealthy Days,Health Outcomes Quality of Life Physical Health % Frequent Physical Distress,Health Outcomes Quality of Life Physical Health Average Number of Physically Unhealthy Days,Health Outcomes Quality of Life STIs HIV Prevalence Rate,INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households Median household income (dollars),INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households With Food Stamp/SNAP benefits in the past 12 months,INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households With Social Security Mean Social Security income (dollars),INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households With retirement income Mean retirement income (dollars),Physical Environment Air and Water Quality Air Pollution Average Daily PM2.5,Physical Environment Housing and Transit Homeownership % Homeowners,Physical Environment Housing and Transit Severe Housing Cost Burden % Severe Housing Cost Burden,Physical Environment Housing and Transit Severe Housing Problems % Severe Housing Problems,Social and Economic Factors Community Safety Firearm Fatalities Firearm Fatalities Rate,Social and Economic Factors Community Safety Homicides Homicide Rate,Social and Economic Factors Community Safety Injury Deaths Injury Death Rate,Social and Economic Factors Community Safety Juvenile Arrests Juvenile Arrest Rate,Social and Economic Factors Community Safety Suicides Suicide Rate (Age-Adjusted),Social and Economic Factors Community Safety Violent Crime Violent Crime Rate,Social and Economic Factors Education Academic Performance Average Grade Performance,Social and Economic Factors Education Disconnected Youth % Disconnected Youth,Social and Economic Factors Education High School Graduation Rate High School Graduation Rate,Social and Economic Factors Education Some College % Some College,Social and Economic Factors Employment Unemployment % Unemployed,Social and Economic Factors Family and Social S

In [70]:
sdoh_fields = ['sdoh_1','sdoh_2','sdoh_3','sdoh_4','sdoh_5','sdoh_6','sdoh_7','sdoh_8']

In [71]:
for column in sdoh_fields:
    df_sdoh_pivot[column].fillna(df_sdoh_pivot.groupby('state_abbrev')[column].transform('mean'), inplace=True)

In [72]:
df_sdoh_pivot['sdoh_avg'] = df_sdoh_pivot[['sdoh_1','sdoh_2','sdoh_3','sdoh_4','sdoh_5','sdoh_6','sdoh_7','sdoh_8']].mean(axis=1)

In [73]:
len(df_sdoh_pivot)

46193

In [74]:
df_sdoh_tableau=df_sdoh_pivot

In [75]:
df_sdoh_tableau.to_csv('df_sdoh_tableau.csv')